In [203]:
import logging; logging.basicConfig(level=logging.INFO)
import tensorflow as tf
import ltn
import pandas as pd

In [204]:
df = pd.read_csv("diabetes.dat")
pd.set_option('display.expand_frame_repr', False)
df = df.sample(frac=1) #shuffle
print(df.head(14))

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  DiabetesPedigreeFunction  Age  Outcome
644            3      103             72             30      152  27.6                     0.730   27        0
659            3       80             82             31       70  34.2                     1.292   27        1
303            5      115             98              0        0  52.9                     0.209   28        1
725            4      112             78             40        0  39.4                     0.236   38        0
297            0      126             84             29      215  30.7                     0.520   24        0
696            3      169             74             19      125  29.9                     0.268   31        1
88            15      136             70             32      110  37.1                     0.153   43        1
20             3      126             88             41      235  39.3                     0.704   27        0
9

In [205]:
features = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]
labels_diabetes = df['Outcome']

batch_size=64
ds_train = tf.data.Dataset.from_tensor_slices((features[:600], labels_diabetes[:600])).batch(batch_size)
ds_test = tf.data.Dataset.from_tensor_slices((features[600:], labels_diabetes[600:])).batch(batch_size)

In [206]:
class MLP(tf.keras.Model):
    """Model that returns logits."""
    def __init__(self, n_classes, hidden_layer_sizes=(16, 16, 8)):
        super(MLP, self).__init__()
        self.denses = [tf.keras.layers.Dense(s, activation="elu") for s in hidden_layer_sizes]
        self.dense_class = tf.keras.layers.Dense(n_classes)

    def call(self, inputs):
        x = inputs[0]
        for dense in self.denses:
            x = dense(x)
        return self.dense_class(x)
    
logits_model = MLP(2)  # Binary classification for diabetes
p = ltn.Predicate.FromLogits(logits_model, activation_function="sigmoid", with_class_indexing=True)



In [207]:
class_diabetes = ltn.Constant(1, trainable=False)  # 1: Diabetes
class_no_diabetes = ltn.Constant(0, trainable=False) # 0: No diabetes

In [208]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2), semantics="forall")
Exists = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMean(p=2),semantics="exists")


In [209]:

formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=2))
five = ltn.Constant(5, trainable=False)
forty = ltn.Constant(40, trainable=False)
thirty = ltn.Constant(30, trainable=False)

# Definindo novos predicados
is_pregnancies_greater_than_five = ltn.Predicate.Lambda(lambda inputs: tf.greater(inputs[:, 0], five.tensor))
is_age_greater_than_forty = ltn.Predicate.Lambda(lambda inputs: tf.greater(inputs[:, 7], forty.tensor))
is_bmi_greater_than_thirty = ltn.Predicate.Lambda(lambda inputs: tf.greater(inputs[:, 5], thirty.tensor))

@tf.function
def axioms(features, labels_diabetes):
    
    x = ltn.Variable("x", features)
    x_diabetes = ltn.Variable("x_diabetes", features[labels_diabetes == 1])
    x_no_diabetes = ltn.Variable("x_no_diabetes", features[labels_diabetes == 0])
    #x_pregnancies = ltn.Variable("x_pregnancies", features[:, 0]) 


    axioms = [
        Forall(x_no_diabetes, p([x_no_diabetes, class_no_diabetes])),
        Forall(x_diabetes, p([x_diabetes, class_diabetes])),
        Forall(x, Not(And(p([x, class_diabetes]), p([x, class_no_diabetes])))),
        #Forall(x, Implies(is_greater_than_five([x_pregnancies]), p([x, class_diabetes])))
    ]
    axioms.append(Exists(x, Implies(is_pregnancies_greater_than_five(x), p([x, class_diabetes]))))
    axioms.append(Forall(x, Implies(is_age_greater_than_forty(x), p([x, class_diabetes]))))
    axioms.append(Forall(x, Implies(is_bmi_greater_than_thirty(x), p([x, class_diabetes]))))
    sat_level = formula_aggregator(axioms).tensor
    return sat_level


In [210]:
for features, labels_diabetes in ds_train:
    print("Initial sat level %.5f" % axioms(features, labels_diabetes))
    break


Initial sat level 0.31724


In [211]:
metrics_dict = {
    'train_sat_kb': tf.keras.metrics.Mean(name='train_sat_kb'),
    'test_sat_kb': tf.keras.metrics.Mean(name='test_sat_kb'),
    'train_accuracy': tf.keras.metrics.Mean(name="train_accuracy"),
    'test_accuracy': tf.keras.metrics.Mean(name="test_accuracy"),
    'test_sat_phi1': tf.keras.metrics.Mean(name='test_sat_phi1'),
    'test_sat_phi2': tf.keras.metrics.Mean(name='test_sat_phi2'),
    #'test_sat_phi3': tf.keras.metrics.Mean(name='test_sat_phi3')
}

In [212]:
@tf.function()
def sat_phi1(features):
    x = ltn.Variable("x",features)
    phi1 = Forall(x, Implies(p([x,class_diabetes]),Not(p([x,class_no_diabetes]))),p=10)
    return phi1.tensor

@tf.function()
def sat_phi2(features):
    x = ltn.Variable("x",features)
    phi2 = Forall(x, Implies(p([x,class_diabetes]),p([x,class_no_diabetes])),p=10)
    return phi2.tensor

#@tf.function()
#def sat_phi3(features):
#    x = ltn.Variable("x", features)
#    phi3 = Forall(x, Implies(is_age_greater_than_forty(x), p([x, class_diabetes])), p=10)
#    return phi3.tensor


def multilabel_hamming_loss(y_true, y_pred, threshold=0.5,from_logits=False):
    if from_logits:
        y_pred = tf.math.sigmoid(y_pred)
    y_pred = y_pred > threshold
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.int32)
    nonzero = tf.cast(tf.math.count_nonzero(y_true-y_pred,axis=-1),tf.float32)
    return nonzero/y_true.get_shape()[-1]

In [213]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
@tf.function
def train_step(features, labels_diabetes):
    # sat and update
    with tf.GradientTape() as tape:
        sat = axioms(features, labels_diabetes)
        loss = 1.-sat
    gradients = tape.gradient(loss, p.trainable_variables)
    optimizer.apply_gradients(zip(gradients, p.trainable_variables))
    metrics_dict['train_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    labels_diabete = (labels_diabetes == 1)
    labels_no_diabete = (labels_diabetes == 0)
    onehot = tf.stack([labels_diabete, labels_no_diabete],axis=-1)
    metrics_dict['train_accuracy'](1-multilabel_hamming_loss(onehot,predictions,from_logits=True))
    
@tf.function
def test_step(features, labels_diabetes):
    # sat
    sat_kb = axioms(features, labels_diabetes)
    metrics_dict['test_sat_kb'](sat_kb)
    metrics_dict['test_sat_phi1'](sat_phi1(features))
    metrics_dict['test_sat_phi2'](sat_phi2(features))
    
    # accuracy
    predictions = logits_model([features])
    labels_no_diabete = (labels_diabetes == 0)
    labels_diabete = (labels_diabetes == 1)
    onehot = tf.stack([labels_no_diabete, labels_diabete], axis=-1)
    metrics_dict['test_accuracy'](1 - multilabel_hamming_loss(onehot, predictions, from_logits=True))

In [1]:
import commons

EPOCHS = 800

commons.train(
    EPOCHS,
    metrics_dict,
    ds_train,
    ds_test,
    train_step,
    test_step,
    csv_path="diabetes_results.csv",
    track_metrics=20
)

NameError: name 'metrics_dict' is not defined